In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### Baseline results - No Preprocessing

In [10]:
# Train and Test variables
x_train = np.array(train.drop(['X10','Y'],axis=1))
y_train = np.array(train['Y'])

x_test = np.array(test.drop(['X10','Y'],axis=1))
y_test = np.array(test['Y'])

# Model creation
model = tf.keras.Sequential([
    tf.keras.Input(shape=(9,)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu,),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(8, activation=tf.nn.relu),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mean_absolute_percentage_error'])
history = model.fit(x_train, y_train, batch_size=32, epochs=20, validation_split = 0.2,verbose=1)
model.evaluate(x_test, y_test, verbose=2)


Epoch 1/20
225/225 [==============================] - 5s 13ms/step - loss: 148.9600 - mean_absolute_percentage_error: 17.2499 - val_loss: 60.9656 - val_mean_absolute_percentage_error: 7.1113
Epoch 2/20
225/225 [==============================] - 2s 8ms/step - loss: 57.6880 - mean_absolute_percentage_error: 6.8829 - val_loss: 56.1490 - val_mean_absolute_percentage_error: 6.5874
Epoch 3/20
225/225 [==============================] - 2s 8ms/step - loss: 56.5647 - mean_absolute_percentage_error: 6.7577 - val_loss: 53.3736 - val_mean_absolute_percentage_error: 6.3066
Epoch 4/20
225/225 [==============================] - 2s 8ms/step - loss: 54.0943 - mean_absolute_percentage_error: 6.4889 - val_loss: 52.8503 - val_mean_absolute_percentage_error: 6.2287
Epoch 5/20
225/225 [==============================] - 2s 8ms/step - loss: 53.2453 - mean_absolute_percentage_error: 6.3930 - val_loss: 51.2492 - val_mean_absolute_percentage_error: 6.2633
Epoch 6/20
225/225 [==============================] - 2s 

[54.6447639465332, 6.332549095153809]

## Actual Exploration

### Data Processing

In [661]:
# Linear Scaling - Help Gradient descent converge quicker towards minima
train_x1_x9 = train.drop(['X10','Y'],axis=1)
train_x1_x9 = (train_x1_x9 - train_x1_x9.min()) / (train_x1_x9.max() - train_x1_x9.min())

test_x1_x9 = test.drop(['X10','Y'],axis=1)
test_x1_x9 = (test_x1_x9 - test_x1_x9.min()) / (test_x1_x9.max() - test_x1_x9.min())

# One Hot Encoding for categorical column 
train_x10 = pd.get_dummies(train['X10'],dtype=int)
test_x10 = pd.get_dummies(test['X10'],dtype=int)

x_train = np.array(pd.concat([train_x1_x9,train_x10],axis=1))
y_train = np.array(train['Y'])

x_test = np.array(pd.concat([test_x1_x9,test_x10],axis=1))
y_test = np.array(test['Y'])

### Hyperparameter Tuning - Grid Search

In [ ]:
# Model creation
def create_model(optimizer,learning_rate):
    model = tf.keras.Sequential([
        keras.Input(shape=(15,)),
        layers.Dense(128,activation=tf.nn.relu),
        layers.Dense(64, activation=tf.nn.relu,),
        layers.Dense(128, activation=tf.nn.relu),
        layers.Dense(64, activation=tf.nn.relu),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(4, activation=tf.nn.relu),
        layers.Dense(1,activation=tf.nn.relu)
    ])
    opt = keras.optimizers.Adam(learning_rate=learning_rate) if optimizer == 'adam' else keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt,loss='mean_absolute_error',metrics=['mean_absolute_percentage_error'])
    return model

model = KerasRegressor(model=create_model,epochs=20,batch_size=128)

param_grid = {
    'model__optimizer': ['adam','sgd'],
    'model__learning_rate': [0.005,0.01,0.05],
    'batch_size': [32,64,128],
    'epochs': [20,30,40]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error',cv=5,verbose=2)
grid_result = grid.fit(x_train,y_train)

In [678]:
print(f"Best Negative MAE: {grid_result.best_score_} using {grid_result.best_params_}")

Best Negative MAE: -9.651480784693238 using {'batch_size': 64, 'epochs': 30, 'model__learning_rate': 0.005, 'model__optimizer': 'adam'}


In [ ]:
# Due to Random nature of NN, need to run a few times to get optimal results
best_model = {'model_mae': 9999}
for i in range(30):
    model = tf.keras.Sequential([
        keras.Input(shape=(15,)),
        layers.Dense(128,activation=tf.nn.relu),
        layers.Dense(64, activation=tf.nn.relu,),
        layers.Dense(128, activation=tf.nn.relu),
        layers.Dense(64, activation=tf.nn.relu),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(4, activation=tf.nn.relu),
        layers.Dense(1,activation=tf.nn.relu)
    ])
    opt = keras.optimizers.Adam(learning_rate=0.005)
    model.compile(optimizer=opt,loss='mean_absolute_error',metrics=['mean_absolute_percentage_error'])
    model.fit(x_train, y_train, batch_size=64, epochs=30, validation_split = 0.2,verbose=0)
    
    model_mae, model_mape = model.evaluate(x_test, y_test, verbose=1)

    if model_mae < best_model['model_mae']:
        best_model['model'] = model
        best_model['model_mae'] = model_mae


In [677]:
print(f"MAE of best model {best_model['model_mae']}")

MAE of best model 6.389989852905273


In [676]:
# Save predictions
preds = pd.DataFrame(best_model['model'].predict(x_test),columns=['y_pred'])
pd.concat([test,preds],axis=1).to_csv('test_pred.csv',index=False)

32/32 [==============================] - 0s 476us/step


In [652]:
# Save best result
best_model['model'].save('best_model.keras')

### Extra Exploration - Random Forest and XGBoost

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [502]:
regr = RandomForestRegressor(n_estimators=100,random_state=42,criterion='absolute_error')
regr.fit(x_train,y_train)

RandomForestRegressor(criterion='absolute_error', random_state=42)

In [665]:
mean_absolute_error(y_test,regr.predict(x_test))

13.999135957161648

In [663]:


clf = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01,random_state=42,eval_metric='mae')
clf.fit(x_train,y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [664]:
mean_absolute_error(y_test,clf.predict(x_test))

10.68182660248541